# Load Libraries and Set CUDA environment

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from model import UNet
from dataloader import ProstateDataset
from utils import ToTensor, plot_images, visualize_predictions, save_model, load_model
import os
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import monai

# Set the environment variable to use GPU 1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Training on GPU.")
else:
    print("CUDA is not available. Training on CPU.")

# Setting up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Dataset and Dataloader

In [4]:
# Initialize your dataset and dataloader
dataset = ProstateDataset(img_dir='./data/img', mask_dir='./data/mask', transform=ToTensor())

# Define the sizes for train, validation, and test sets
train_size = int(0.7 * len(dataset))  # 70% for training
valid_size = int(0.2 * len(dataset))  # 20% for validation
test_size = len(dataset) - train_size - valid_size  # Remaining 10% for testing

# Split the dataset
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# Create DataLoaders for each split
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)

In [20]:
# Initialize model
model = UNet(n_channels=1, n_classes=1).to(device)
# load model
load_model(model, 'model_unet.pth', device)
model = model.eval()  # Set model to evaluation mode

Model loaded from model_unet.pth


## Dice Score Calculation

##
The Dice coefficient measures the similarity between two sets (e.g., predicted and ground truth masks) and is defined as:
$$ \text{Dice} = \frac{2 \times |A \cap B|}{|A| + |B|} $$

The value ranges from 0 (no overlap) to 1 (perfect overlap), making it ideal for segmentation tasks.

For segmentation evaluation we can use other metrics such as - 
Intersection over Union (IoU or Jaccard Index), Hausdorff Distance (Boundary-Based)

In [16]:
# test evaluation
test_dice = 0.0

with torch.no_grad():
    for batch in test_loader:
        images = batch['image'].to(device)
        true_masks = batch['mask'].to(device)
        preds = torch.sigmoid(model(images))
        preds = (preds > 0.5).float()
        # Calculate Dice coefficient
        monai_dice = monai.metrics.DiceMetric(include_background=False, ignore_empty=False,  reduction='mean')
        monai_dice(y_pred=preds, y=true_masks)
        test_dice += monai_dice.aggregate().item()
        monai_dice.reset()
    test_dice /= len(test_loader)
    print(f'Test Dice Coefficient: {test_dice:.4f}')

Test Dice Coefficient: 0.9135
